# Area Processing (Tutorial 2)
***
# Table of Contents
1.   [Imports](#Imports)
2.   [Exercise 1 - Sliding Window](#Exercise-1---Sliding-Window)
3.   [Exercise 2 - Convolution on RoI](#Exercise-2---Convolution-on-RoI)
4.   [Exercise 3 - Convolution on the Whole Image](#Exercise-3---Convolution-on-the-Whole-Image)
5.   [Exercise 4 - Different Convolution Kernels](#Exercise-4---Different-Convolution-Kernels)


# Imports

Only 3 libraries are needed for this project:
* opencv (cv2) - For image processing
* numpy - For its arrays
* matplotlib - Plotting histograms

In [1]:

import cv2
import numpy as np
from matplotlib import pyplot as plt


# Exercise 1 - Sliding Window

### Code Explenation

First I load in the image i will be using for this tutorial and apply a gray filter to it.

In [2]:
original_image = cv2.imread("cursed.jpg")
height, width, _ = original_image.shape
gray = cv2.cvtColor(original_image, cv2.COLOR_BGR2GRAY)

### cursed.jpg

The image I will be using for this lab is one of my failed attempts of creating a pie chart for one of my statistics units
from last year. It features a sun like object in the centre with black lines coming out from the centre, it is coloured in
full RGB while the background is gray.

<img src="cursed.jpg">

In [7]:
class Window(object):
    def __init__(self, image, n, s):
        # make s can be tuple
        self.x_boundary = image.shape[1] + n
        self.y_boundary = image.shape[0] + s
        self.top_left = (0, 0)
        self.bot_right = (n, n)
        self.previousBotY = n
        self.height = n
        self.stride = (s, s)

    def forwardPos(self):
        # Case when you need to go down and start new line
        if (self.bot_right + self.stride)[0] >= (self.x_boundary - self.height):
           # print("down")
            return (0, self.top_left[1] + self.stride[1]), (self.height, self.bot_right[1] + self.stride[1])
        # generic move right case
        else:
            return (self.top_left[0] + self.stride[0], self.top_left[1]), (self.bot_right[0] + self.stride[0], self.bot_right[1])

    def forwardMove(self):
        self.top_left, self.bot_right = self.forwardPos()
        return self.top_left, self.bot_right

    def backwardPos(self):
        # Case when you need to go up and go end of line
        if (self.top_left - self.stride)[0] < 0:
            return (self.x_boundary - self.height, self.top_left[1] - self.stride[1]), (self.x_boundary, self.bot_right[1] - self.stride[1])
        # generic move left case
        else:
            return (self.top_left[0] - self.stride[0], self.top_left[1]), (self.bot_right[0] - self.stride[0], self.bot_right[1])

    def backwardMove(self):
        self.top_left, self.bot_right = self.backwardPos()
        return self.top_left, self.bot_right

    def inBoundary(self, new_top_left=None, new_bot_right=None):
        if new_top_left is None:
            new_top_left =  self.top_left
        if new_bot_right is None:
            new_bot_right =  self.bot_right

        return  new_bot_right[0] <= self.x_boundary and new_bot_right[1] <= self.y_boundary and \
                new_top_left[0] >= 0 and new_top_left[1] >= 0

    def changedY(self):
        if self.previousBotY == self.bot_right[1]:
            return False
        else:
            self.previousBotY = self.bot_right[1]
            return True

    # add get image in boundary
    def getImageInBoundary(self, image):
        safety = np.zeros((self.height, self.height))
        new_image = []
        for i in range(self.bot_right[1]):
            new_image.append(image[:self.top_left[1] + i + 1][self.top_left[1]][self.top_left[0]:self.bot_right[1]])
        # see how to add 2 different shapes with numpy
        return safety + np.array(new_image)


    def __str__(self):
        return "Top Left Corner " + str(self.top_left) + "\nBot Right Corner " + str(self.bot_right)

    def getPos(self):
        return self.top_left, self.bot_right

In [8]:
%%capture
win = Window(gray, 500, 100)
start_point, end_point = win.getPos()

# White
color = (255, 255, 255)

image = cv2.rectangle(gray.copy(), start_point, end_point, color)

cv2.imwrite("Output/GrayWithRectangle.png", image, [cv2.IMWRITE_PNG_COMPRESSION, 0])
image2 = win.getImageInBoundary(gray)
cv2.imwrite("Output/ROI.png", image2, [cv2.IMWRITE_PNG_COMPRESSION, 0])

In [5]:
%%capture
show = True

new_tl, new_br = win.forwardPos()
while win.inBoundary(new_br):
    image = cv2.rectangle(gray.copy(), start_point, end_point, (255, 255, 255))
    if show:
        cv2.imshow("Sliding Window", image)
        cv2.waitKey(int(1/35*1000))
    start_point, end_point = win.forwardMove()
    new_tl, new_br = win.forwardPos()
cv2.destroyAllWindows()
cv2.imwrite("Output/LastPic.png", image, [cv2.IMWRITE_PNG_COMPRESSION, 0])


# Exercise 2 - Convolution on RoI

In [10]:
class Sobel(object):
    def __init__(self, x=None, y=None):
        self.x = np.array([[-1, 0, 1],
                           [-2, 0, 2],
                           [-1, 0, 1]]) \
                    if x is None else x
        self.y = np.array([ [-1, -2, -1],
                            [0, 0, 0],
                            [1, 2, 1]]) \
                    if y is None else y

    def filter(self, roi, axis=0):
        """
        :param roi: image that will have the filter applied to it
        :param axis: 0 is x, 1 is y
        :return:
        """
        kernel = self.x if axis == 0 else self.y

        _filter = kernel * roi
        sum_of_filter = _filter.sum()

        return sum_of_filter

    def filterImage(self, image, window=None, axis=0):
        new_roi = []
        line = []
        if window is None:
            # go over entire image
            moving_kernel = Window(image, 3, 1)
        else:
            image = window.getImageInBoundary(image)
            moving_kernel = Window(image, 3, window.stride[0])

        new_tl, new_br = moving_kernel.forwardPos()
        while moving_kernel.inBoundary(new_br):
            if moving_kernel.changedY():
                new_roi.append(line)
                line = []
            else:
                roi = moving_kernel.getImageInBoundary(image)
                line.append(self.filter(roi, axis))

            moving_kernel.forwardMove()
            new_tl, new_br = moving_kernel.forwardPos()

        return np.array(new_roi)

sobel = Sobel()

win = Window(gray, 30, 1)

image = gray.copy()
roi = win.getImageInBoundary(image)
cv2.imwrite("Output/roi_before_filter.png", roi, [cv2.IMWRITE_PNG_COMPRESSION, 0])
filtered = sobel.filterImage(image, win)
cv2.imwrite("Output/roi_after_filter.png", filtered, [cv2.IMWRITE_PNG_COMPRESSION, 0])
#test = sobel.filter(image, win)

ValueError: operands could not be broadcast together with shapes (3,3) (3,2) 

# Exercise 3 - Convolution on the Whole Image

# Exercise 4 - Different Convolution Kernels